# Multicriterio


In [69]:
import numpy as np
import pandas as pd

xls=pd.ExcelFile(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\imdbmoviesdata.xlsx")
datos=pd.read_excel(xls,"tmdb_movies_data")
tabla_AHP=pd.read_excel(xls,"AHP")
tabla_AHP_gen=pd.read_excel(xls,"AHP_genero")
tabla_AHP_time=pd.read_excel(xls,"AHP_duracion")


xls2=pd.ExcelFile(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\AHP_multicriterio.xlsx")
datos2=pd.read_excel(xls2,"Datos")
t_AHP=pd.read_excel(xls2,"AHP")
AHP_tipo=pd.read_excel(xls2,"AHP_color")
#[65,90,120,180,340]


In [34]:
#Implementación con más columnas categóricas
cols=datos.columns
num_cols=datos._get_numeric_data().columns
categ_cols=list(set(cols)-{"Nombre"}-set(num_cols))

valores_prueba=list(tabla_AHP_gen.iloc[:,0])

tabla_AHP_gen[valores_prueba]#.values
values=list(range(16))#[0.1,0.2,0.3,0.4,0.5]
dic_prueba=dict(zip(valores_prueba,values))
datos["Género"]=datos["Género"].replace(dic_prueba)
datos

,Nombre,Popularidad,Presupuesto,Ventas,Duración,Género,Votos,PromedioVotos,AñoEstreno
0,Mission: Impossible - Rogue Nation,4.566713,150000000,682330139,131,0,2349,7.1,2015
1,The Guardian,0.632875,70000000,55011732,139,0,212,7.0,2006
2,Only the Strong,0.207999,6000000,3283371,99,0,12,6.7,1993
3,State of Play,1.043738,60000000,87784194,127,0,257,6.6,2009
4,Bloodsport,1.392581,1500000,11806119,92,0,180,6.4,1988
...,...,...,...,...,...,...,...,...,...
3591,The Homesman,0.870104,16000000,2429989,122,15,150,6.2,2014
3592,Seraphim Falls,0.463068,18000000,1196307,115,15,65,5.8,2006
3593,The Revenant,9.110700,135000000,532950503,156,15,3929,7.2,2015
3594,The Long Riders,0.128234,10000000,15795189,99,15,17,6.0,1980


In [82]:
def prepos_data(datos:pd.DataFrame,vect_max_min):
    """Recibe:
    datos: base de datos
    vect_max_min: vector con tantas entradas como variables numéricas y en orden de aparición en la base de datos.
    1 si es de maximización, 0 si es de minimización.
    Retorna:
    Base de datos normalizada tomando en cuenta objetivo (min o max)"""
    datos_num=datos.select_dtypes(include="number")
    for i in range(len(vect_max_min)):
        if vect_max_min[i]==1:
            datos_num.iloc[:,i]=(datos_num.iloc[:,i]-datos_num.iloc[:,i].min())/(datos_num.iloc[:,i].max()-datos_num.iloc[:,i].min())
        else: 
            datos_num.iloc[:,i]=(datos_num.iloc[:,i].max()-datos_num.iloc[:,i])/(datos_num.iloc[:,i].max()-datos_num.iloc[:,i].min())
    datos[datos_num.columns]=datos_num #Asignación sobre el mismo objeto DataFrame

def calc_ponder(matr:np.array,tol:float):
    """Recibe:
    matr: matriz nxn de comparación AHP dos a dos recíproca.
    tol: tolerancia de variación entre vectores suma normalizados.
    Retorna:
    vect_pond: vector ponderación. Eleva error si no se cumple consistencia"""
    n,_=matr.shape
    epsilon=np.array([tol+1])
    i=2
    prev_v_norm=matr.sum(axis=1)/matr.sum()
    while (epsilon>=tol).any():
        if i>6:
            raise Exception("Matriz no consistente por iteraciones")
        A_n=np.linalg.matrix_power(matr,i)#Eleva error si no es cuadrada
        v_norm=A_n.sum(axis=1)/A_n.sum() #vector de suma de filas normalizado
        epsilon=max(v_norm-prev_v_norm)
        prev_v_norm=v_norm
    print("\n v_norm", v_norm)
    #Verificación consistencia
    v1=np.matmul(matr,v_norm)
    v1_perc=np.divide(v1,v_norm)
    print("\n v1,v1_perc\n",v1,v1_perc)
    lambda_max=np.mean(v1_perc)
    print(lambda_max)
    indice_RI={1:0,3:0.58,4:0.89,5:1.11,6:1.24,7:1.32,8:1.4,9:1.5,10:1.49}
    RI=indice_RI[n]
    CI=(lambda_max-n)/(n-1)
    CR=CI/RI
    if CR>0.1:
        raise Exception("Matriz no consistente por prueba")
    return v_norm

def categ_AHP(tabla_AHP_cat:pd.DataFrame)->dict:
    """Recibe:
    tabla_AHP_cat: tabla comparativa sobre los posibles valores de una variable (criterio) categórica
    Retorna:
    Diccionario con valores:ponderaciónAHP"""
    
    valores=list(tabla_AHP_cat.iloc[:,0])
    matriz=tabla_AHP_cat[valores].values
    vect_pond=calc_ponder(matriz,0.05)
    dicc_valor_ponderacion=dict(zip(valores,vect_pond))
    return dicc_valor_ponderacion

    

    
def AHP(datos,tabla_AHP,vect_max_min,*tablas_AHP_cat):
    """Recibe:
    datos: base de datos de alternativas
    tabla_AHP: matriz recíproca sobre cada criterio.
    vect_max_min: vector con tantas entradas como variables numéricas y especificando con orden respectivo
    a la base de datos: 0 si es minimización, 1 si es maximización"""
    prepos_data(datos,vect_max_min) #Altera el objeto datos
    valores_vab=list(tabla_AHP.iloc[:,0])
    matriz_AHP=tabla_AHP[valores_vab].values
    v_ponderac_crit=calc_ponder(matriz_AHP,0.05)
    for tabla_cat in tablas_AHP_cat:
        criterio=tabla_cat.columns[0]#La tabla debe tener como primer elemento 
        #del encabezado el nombre del criterio corespondiente y como primera columna, los valores que puede tomar
        dicc_valorac=categ_AHP(tabla_cat)
        datos[criterio]=datos[criterio].replace(dicc_valorac)
    #norm_data=



#datos["Género"].unique()
#matr=np.array([[1,3,5,3,5],[0.33,1,2,1,3],[0.2,0.5,1,1,2],[0.33,1,1,1,4],[0.2,0.33,0.5,0.25,1]])

#matr=np.array([[1,3,0.25,5,6],[1/3,1,0.5,2,4],[4,2,1,6,9],[0.2,0.5,1/6,1,3],[1/6,0.25,1/9,1/3,1]])
#matr=np.array([[1,0.5,1,3],[2,1,8,9],[1,0.125,1,2],[1/3,1/9,1/2,1]])
#calc_ponder(matr,0.005)

In [83]:
AHP(datos2,t_AHP,[1,1,0,1],AHP_tipo)


 v_norm [0.26854363 0.13837808 0.48697035 0.07099429 0.03511365]

 v1,v1_perc
 [1.3710738  0.75382097 2.57988963 0.38039472 0.19223802] [5.10559039 5.4475463  5.29783719 5.35810277 5.47473806]
5.336762941455308
Se logró? [0.26854363 0.13837808 0.48697035 0.07099429 0.03511365]

 v_norm [0.19428693 0.62197255 0.12221103 0.06152949]

 v1,v1_perc
 [0.81207271 2.54200011 0.51730351 0.25650538] [4.17975996 4.08699727 4.23287077 4.16881998]
4.167111996093883


In [71]:
t_AHP.iloc[:,0]

0       RAM
1        Gb
2    Precio
3    Cámara
4     Color
Name: Unnamed: 0, dtype: object

In [5]:
tabla_AHP_gen.head()

,Unnamed: 0,Action,Adventure,Animation,Comedy,Crime,Drama,Familiy,Fantasy,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
0,Action,1.000000,2.000000,1.500000,3.0,4.0,1.5,4.0,3.0,5.0,2.0,3.0,1.5,2.0,1.5,1.5,2.0
1,Adventure,0.500000,1.000000,0.900000,1.5,2.0,1.0,2.0,1.0,3.0,1.0,2.0,0.5,1.0,1.5,0.8,1.0
2,Animation,0.666667,1.111111,1.000000,2.0,3.0,1.0,3.0,2.0,4.0,1.5,2.0,1.0,1.5,1.0,1.0,1.5
3,Comedy,0.333333,0.666667,0.500000,1.0,2.0,0.5,3.0,1.0,3.0,0.5,1.0,0.5,0.8,0.5,0.5,0.8
4,Crime,0.250000,0.500000,0.333333,0.5,1.0,0.3,2.0,0.8,2.0,0.3,0.5,0.4,0.5,0.3,0.3,0.5
